In [ ]:
import keras
from keras.layers import *
from keras.activations import *
from keras.models import *
from keras.utils import *
import keras.backend as K
from keras.initializers import glorot_uniform
from keras.preprocessing import image
from keras.datasets import mnist
from keras.losses import *
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, Callback
import tensorflow as tf

import numpy as np
import os
import random
from timeit import default_timer as timer
import time
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

In [ ]:
# (756, 434)
# (252, 145)
img_dim = (145, 252)
img_shape = (145, 252, 3)
epoch_number = 50
batches = 16

path = "./full_dataset/"
if not os.path.exists(path):
    os.makedirs(path)

**Base dataset**
---

In [ ]:
# Callback class for time history
class TimingCallback(keras.callbacks.Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

In [ ]:
def get_model(img_shape, classes=4):
  # Implementing LeNet-5 with Keras functional API
  input_img = keras.Input(shape=(img_shape))

  x = Conv2D(filters=6, kernel_size=5, strides=1, activation='relu')(input_img)  # conv1
  x = Dropout(0.2)(x)  
  x = MaxPooling2D(pool_size=2, strides=2)(x)  # pooling1

  x = Conv2D(filters=16, kernel_size=5, strides=1, activation='relu')(x)  # conv2
  x = Dropout(0.2)(x)
  x = MaxPooling2D(pool_size=2, strides=2)(x)  # pooling2

  x = Flatten()(x)  # Flatten
  x = Dense(units=120, activation='relu')(x)  # FCC1
  x = Dense(units=84, activation='relu')(x)  # FCC2
  class_prob = Dense(units = classes, activation='softmax')(x)  # FCC3/output

  model_train = Model(input_img, class_prob)
  
  return model_train

In [ ]:
def identity_block(X, f, filters, stage, block):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value. We'll need this later to add back to the main path. 
    X_shortcut = X

    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

def convolutional_block(X, f, filters, stage, block, s = 2):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    
    ##### SHORTCUT PATH ####
    X_shortcut = Conv2D(F3, (1, 1), strides = (s,s), name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

def get_resnet_model(input_shape=img_shape, classes=4):   
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    # X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    # X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    # X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    # X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    # X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    # X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    # X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    # X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # Avg pool.
    X = GlobalAveragePooling2D(name='global_avg_pool')(X)

    # output layer
    # X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet')

    return model

In [ ]:
datagen = image.ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    directory='../input/trafficclassified/archive/traffic classified/full_dataset',
    target_size=img_dim,
    color_mode='rgb',
    class_mode='categorical',
    subset='training')

validation_generator = datagen.flow_from_directory(
    directory='../input/trafficclassified/archive/traffic classified/full_dataset',
    target_size=img_dim,
    color_mode='rgb',
    class_mode='categorical',
    subset='validation')

In [ ]:
model_base = get_model(img_shape, 4)
# model_base = get_resnet_model((img_shape), 4)
model_base.summary()

In [ ]:
tf.keras.utils.plot_model(model_base, show_dtype=True, show_layer_names=True, show_shapes=True, expand_nested=True, to_file='model_plot.png')

In [ ]:
# Training
time_callback = TimingCallback()
model_base.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
history_base = model_base.fit(train_generator,
                              epochs=epoch_number,
                              steps_per_epoch=train_generator.n//train_generator.batch_size,
                              validation_data=validation_generator,
                              validation_steps=validation_generator.n//validation_generator.batch_size,
                              callbacks = [time_callback])

In [ ]:
# Checking that I got the values
print(history_base.history.keys())
print(history_base.history['loss'])
print(history_base.history['accuracy'])
print(history_base.history['val_loss'])
print(history_base.history['val_accuracy'])

In [ ]:
# Saving weights
model_base.save_weights(path + 'base_weights.h5')

In [ ]:
base_time = time_callback.logs
print(np.mean(base_time), base_time)

In [ ]:
# Plotting
fig, (ax1, ax2) = plt.subplots(1, 2, sharex=True, figsize=(25, 10))

ax1.plot(history_base.history['accuracy'])
ax1.plot(history_base.history['val_accuracy'])
ax1.set_title('Accuracy over epochs')
ax1.set(xlabel='Epoch', ylabel='Accuracy')
ax1.grid()
ax1.legend(['Train', 'Validation'], loc='lower right')

ax2.plot(history_base.history['loss'])
ax2.plot(history_base.history['val_loss'])
ax2.set_title('Loss over epochs')
ax2.set(xlabel='Epoch', ylabel='Loss')
ax2.grid()
ax2.legend(['Train', 'Validation'], loc='upper right')


plt.show()
fig.savefig(path + 'Loss and Accuracy over Epochs (Base).png')

**Reduced dataset**
---


In [ ]:
path = "./reduced_dataset/"
if not os.path.exists(path):
    os.makedirs(path)

In [ ]:
datagen = image.ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    directory='../input/trafficclassified/archive/traffic classified/reduced_dataset',
    target_size=img_dim,
    color_mode='rgb',
    class_mode='categorical',
    subset='training')

validation_generator = datagen.flow_from_directory(
    directory='../input/trafficclassified/archive/traffic classified/reduced_dataset',
    target_size=img_dim,
    color_mode='rgb',
    class_mode='categorical',
    subset='validation')

In [ ]:
model_reduced = get_model(img_shape, 4)
# model_reduced = get_resnet_model(img_shape, 4)
model_reduced.summary()

In [ ]:
# Training
time_callback = TimingCallback()
model_reduced.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
history_reduced = model_reduced.fit(train_generator,
                              epochs=epoch_number,
                              steps_per_epoch=train_generator.n//train_generator.batch_size,
                              validation_data=validation_generator,
                              validation_steps=validation_generator.n//validation_generator.batch_size,
                              callbacks = [time_callback])

In [ ]:
# Checking that I got the values
print(history_reduced.history.keys())
print(history_reduced.history['loss'])
print(history_reduced.history['accuracy'])
print(history_reduced.history['val_loss'])
print(history_reduced.history['val_accuracy'])

In [ ]:
# Saving weights
model_reduced.save_weights(path + 'reduced_weights.h5')

In [ ]:
reduced_time = time_callback.logs
print(np.mean(reduced_time), reduced_time)

In [ ]:
# Plotting
fig, (ax1, ax2) = plt.subplots(1, 2, sharex=True, figsize=(25, 10))

ax1.plot(history_reduced.history['accuracy'])
ax1.plot(history_reduced.history['val_accuracy'])
ax1.set_title('Accuracy over epochs')
ax1.set(xlabel='Epoch', ylabel='Accuracy')
ax1.grid()
ax1.legend(['Train', 'Validation'], loc='lower right')

ax2.plot(history_reduced.history['loss'])
ax2.plot(history_reduced.history['val_loss'])
ax2.set_title('Loss over epochs')
ax2.set(xlabel='Epoch', ylabel='Loss')
ax2.grid()
ax2.legend(['Train', 'Validation'], loc='upper right')


plt.show()
fig.savefig(path + 'Loss and Accuracy over Epochs (Reduced).png')

**Time difference**
---


In [ ]:
print(np.mean(base_time))
print(np.mean(reduced_time))

**To-Do:**
---

* og_dataset, reduced_dataset (less number of data, hard code the time for LSH for each image)
* Test with both datasets (compare time)
* Stopping time based on accuracy > 98%
* Streaming data
* Compare with different data reduction methods

Other datasets: (feel free to add datasets here)
* cifar10
* stanford dataset?
* pandaset